In [1]:
# NHSRC PHC SUPPLY CHAIN - SIMULATION, RISK & GOVERNANCE ENGINE
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("🛡️ NHSRC PHC SIMULATION & RISK GOVERNANCE ENGINE")
print("=" * 70)

# 1️⃣ LOAD EXISTING DATA LAYERS
print("📥 1. Loading Multi-Layer Data Inputs...")

# Load replenishment recommendations (Day 6 output)
replenishment = pd.read_csv("reports/replenishment_recommendations.csv")

# Load forecast-ready time series for historical patterns
df_ts = pd.read_csv("data/forecast_ready_timeseries.csv")
df_ts['date'] = pd.to_datetime(df_ts['date'], dayfirst=True, errors='coerce')

# Load stock health matrix for NHSRC metrics
stock_health = pd.read_csv("reports/stock_health_matrix.csv")

# Load best model selections
best_models = pd.read_csv("reports/best_model_selection.csv")

print(f"   Replenishment data: {len(replenishment)} SKUs")
print(f"   Time series data: {len(df_ts):,} records")
print(f"   Historical range: {df_ts['date'].min().strftime('%Y-%m-%d')} to {df_ts['date'].max().strftime('%Y-%m-%d')}")

# 2️⃣ BUILD STOCK PROJECTION SIMULATION (14-DAY HORIZON)
print("\n📊 2. Building 14-Day Stock Projection Simulation...")

def simulate_stock_projection(row):
    """Simulate stock levels over next 14 days"""
    current_stock = row['current_stock']
    forecast_14d = row['forecast_14d']
    stock_in_transit = row.get('stock_in_transit', 0)
    lead_time_days = row['lead_time_days']
    MSL = row.get('MSL', 0)
    
    # Daily forecast (assuming linear consumption)
    daily_forecast = forecast_14d / 14 if forecast_14d > 0 else 0
    
    # Simulate 14 days
    stock_levels = []
    stockout_flags = []
    
    stock = current_stock
    
    for day in range(1, 15):
        # Daily consumption
        stock -= daily_forecast
        
        # Add stock in transit if lead time reached
        if day == lead_time_days and stock_in_transit > 0:
            stock += stock_in_transit
        
        # Record metrics
        stock_levels.append(stock)
        stockout_flags.append(1 if stock < 0 else 0)
    
    # Calculate key metrics
    min_stock = min(stock_levels)
    stockout_days = sum(stockout_flags)
    avg_stock = np.mean([s for s in stock_levels if s > 0])
    
    # Excess risk: stock > 3× MSL
    excess_risk = 1 if avg_stock > (MSL * 3) else 0
    
    return {
        'min_stock_14d': min_stock,
        'stockout_days': stockout_days,
        'avg_stock_14d': avg_stock,
        'excess_risk_flag': excess_risk,
        'projected_stock_end': stock_levels[-1]
    }

# Apply simulation to each SKU
print("   Running stock projections SKU-by-SKU...")
simulation_results = []

for idx, row in replenishment.iterrows():
    sku_id = row['sku_id']
    
    # Get additional data from time series
    sku_ts = df_ts[df_ts['sku_id'] == sku_id]
    stock_in_transit = sku_ts['stock_in_transit'].iloc[-1] if 'stock_in_transit' in sku_ts.columns and len(sku_ts) > 0 else 0
    
    # Prepare simulation row
    sim_row = row.copy()
    sim_row['stock_in_transit'] = stock_in_transit
    
    # Run simulation
    sim_output = simulate_stock_projection(sim_row)
    
    # Combine with SKU info
    sim_output['sku_id'] = sku_id
    sim_output['sku_name'] = row['sku_name']
    sim_output['ved_category'] = row['ved_category']
    sim_output['current_stock'] = row['current_stock']
    sim_output['forecast_14d'] = row['forecast_14d']
    
    simulation_results.append(sim_output)

simulation_df = pd.DataFrame(simulation_results)
print(f"✅ Simulated projections for {len(simulation_df)} SKUs")

# 3️⃣ GENERATE SCENARIO OUTCOMES
print("\n🌍 3. Generating Risk Scenarios...")

scenarios = {
    'Base': {'lead_time_factor': 1.0, 'demand_factor': 1.0, 'description': 'Normal planning conditions'},
    'Outbreak': {'lead_time_factor': 1.0, 'demand_factor': 1.8, 'description': 'Epidemic/seasonal surge (80% demand increase)'},
    'Supply_Delay': {'lead_time_factor': 1.6, 'demand_factor': 1.1, 'description': 'Transportation disruption'},
    'Worst_Case': {'lead_time_factor': 2.0, 'demand_factor': 2.5, 'description': 'Catastrophic risk detection'}
}

scenario_results = []

for scenario_name, params in scenarios.items():
    print(f"   Simulating: {scenario_name} ({params['description']})")
    
    for idx, row in replenishment.iterrows():
        sku_id = row['sku_id']
        
        # Apply scenario factors
        scenario_forecast = row['forecast_14d'] * params['demand_factor']
        scenario_lead_time = row['lead_time_days'] * params['lead_time_factor']
        
        # Calculate scenario-specific metrics
        daily_forecast = scenario_forecast / 14
        days_until_stockout = row['current_stock'] / daily_forecast if daily_forecast > 0 else 999
        
        # Risk flags
        stockout_risk = 1 if days_until_stockout < scenario_lead_time else 0
        
        # Get expiry risk from time series (if available)
        sku_ts = df_ts[df_ts['sku_id'] == sku_id]
        if 'expiry_days_remaining' in sku_ts.columns and len(sku_ts) > 0:
            min_expiry = sku_ts['expiry_days_remaining'].min()
            expiry_risk = 1 if min_expiry < 30 and row['current_stock'] > (row['ADC'] * 30) else 0
        else:
            expiry_risk = 0
        
        scenario_results.append({
            'scenario': scenario_name,
            'sku_id': sku_id,
            'sku_name': row['sku_name'],
            'ved_category': row['ved_category'],
            'demand_factor': params['demand_factor'],
            'lead_time_factor': params['lead_time_factor'],
            'scenario_forecast_14d': scenario_forecast,
            'days_until_stockout': days_until_stockout,
            'stockout_risk_flag': stockout_risk,
            'expiry_risk_flag': expiry_risk,
            'severity_score': stockout_risk * 10 + expiry_risk * 5
        })

scenario_summary = pd.DataFrame(scenario_results)

# 4️⃣ ASSIGN RESILIENCE SCORE (0-100)
print("\n🛡️ 4. Calculating Resilience Scores...")

def calculate_resilience_score(row, scenario_data):
    """Calculate comprehensive resilience score (0-100)"""
    score = 100
    
    # 1. Stockout risk component (40 points max)
    sku_scenarios = scenario_data[scenario_data['sku_id'] == row['sku_id']]
    stockout_scenarios = sku_scenarios['stockout_risk_flag'].sum()
    stockout_penalty = (stockout_scenarios / len(scenarios)) * 40
    score -= stockout_penalty
    
    # 2. Expiry risk component (30 points max)
    expiry_scenarios = sku_scenarios['expiry_risk_flag'].sum()
    expiry_penalty = (expiry_scenarios / len(scenarios)) * 30
    score -= expiry_penalty
    
    # 3. Demand volatility component (20 points max)
    demand_volatility = row.get('ADC_std', 0) / row.get('ADC', 1) if row.get('ADC', 0) > 0 else 1.0
    volatility_penalty = min(demand_volatility * 20, 20)
    score -= volatility_penalty
    
    # 4. Coverage safety component (10 points max)
    coverage_ratio = row.get('days_cover', 0) / row.get('lead_time_days', 1)
    if coverage_ratio < 1:
        coverage_penalty = (1 - coverage_ratio) * 10
        score -= min(coverage_penalty, 10)
    
    return max(0, min(100, score))

# Calculate resilience scores
resilience_scores = []
for idx, row in replenishment.iterrows():
    resilience = calculate_resilience_score(row, scenario_summary)
    resilience_scores.append({
        'sku_id': row['sku_id'],
        'sku_name': row['sku_name'],
        'ved_category': row['ved_category'],
        'resilience_score': resilience
    })

resilience_df = pd.DataFrame(resilience_scores)

# 5️⃣ ESCALATION BUCKET CLASSIFICATION
print("\n🚨 5. Assigning Escalation Buckets...")

def classify_escalation(score):
    """Map resilience score to escalation bucket"""
    if score >= 90:
        return 'Stable', 'Monitor'
    elif score >= 70:
        return 'Watchlist', 'Increase buffer'
    elif score >= 40:
        return 'Risky', 'Reorder or redistribute'
    else:
        return 'Critical', 'Immediate action + emergency procurement'

escalation_data = []
for idx, row in resilience_df.iterrows():
    risk_status, required_action = classify_escalation(row['resilience_score'])
    
    # Get key reasons from simulation
    sku_scenarios = scenario_summary[scenario_summary['sku_id'] == row['sku_id']]
    stockout_count = sku_scenarios['stockout_risk_flag'].sum()
    expiry_count = sku_scenarios['expiry_risk_flag'].sum()
    
    # Determine key reason
    if stockout_count >= 2:
        key_reason = f"Stockout risk in {stockout_count} scenarios"
    elif expiry_count >= 2:
        key_reason = f"Expiry risk in {expiry_count} scenarios"
    elif row['resilience_score'] < 40:
        key_reason = "High demand volatility + low coverage"
    else:
        key_reason = "Moderate risk across multiple dimensions"
    
    # Confidence level based on data quality
    sku_ts = df_ts[df_ts['sku_id'] == row['sku_id']]
    confidence = "HIGH" if len(sku_ts) >= 100 else "MEDIUM" if len(sku_ts) >= 30 else "LOW"
    
    escalation_data.append({
        'sku_id': row['sku_id'],
        'sku_name': row['sku_name'],
        'ved_category': row['ved_category'],
        'resilience_score': row['resilience_score'],
        'risk_status': risk_status,
        'required_action': required_action,
        'key_reason': key_reason,
        'confidence_level': confidence
    })

escalation_df = pd.DataFrame(escalation_data)

# 6️⃣ ALERTS FILE EXPORT
print("\n📤 6. Exporting Simulation Outputs...")

# Export scenario summary
scenario_summary_path = "reports/scenario_summary.csv"
scenario_summary.to_csv(scenario_summary_path, index=False)
print(f"   ✅ {scenario_summary_path}")

# Export simulation dashboard
dashboard_data = pd.merge(simulation_df, escalation_df, on='sku_id')
dashboard_path = "reports/simulation_dashboard.csv"
dashboard_data.to_csv(dashboard_path, index=False)
print(f"   ✅ {dashboard_path}")

# Export alerts as JSON
alerts = []
for idx, row in escalation_df.iterrows():
    if row['risk_status'] in ['Critical', 'Risky']:
        alert = {
            'sku': row['sku_id'],
            'sku_name': row['sku_name'],
            'risk_status': row['risk_status'],
            'key_reason': row['key_reason'],
            'recommended_action': row['required_action'],
            'confidence_level': row['confidence_level'],
            'timestamp': datetime.now().isoformat(),
            'escalation_level': 'HIGH' if row['risk_status'] == 'Critical' else 'MEDIUM'
        }
        alerts.append(alert)

alerts_path = "reports/system_alerts.json"
with open(alerts_path, 'w') as f:
    json.dump(alerts, f, indent=2)
print(f"   ✅ {alerts_path} ({len(alerts)} alerts generated)")

# 7️⃣ RISK DISTRIBUTION ANALYSIS
print("\n📊 7. Risk Distribution Analysis...")

risk_distribution = escalation_df['risk_status'].value_counts()
print("\n   RISK STATUS DISTRIBUTION:")
for status, count in risk_distribution.items():
    percentage = (count / len(escalation_df)) * 100
    print(f"   - {status}: {count} SKUs ({percentage:.1f}%)")

# Scenario impact analysis
print("\n   SCENARIO IMPACT ANALYSIS:")
for scenario in scenarios.keys():
    scenario_risks = scenario_summary[scenario_summary['scenario'] == scenario]
    stockout_count = scenario_risks['stockout_risk_flag'].sum()
    expiry_count = scenario_risks['expiry_risk_flag'].sum()
    print(f"   - {scenario}: {stockout_count} stockout risks, {expiry_count} expiry risks")

# VED category risk analysis
print("\n   VED CATEGORY RISK PROFILE:")
for ved in ['Vital', 'Essential', 'Desirable']:
    ved_skus = escalation_df[escalation_df['ved_category'] == ved]
    if len(ved_skus) > 0:
        avg_score = ved_skus['resilience_score'].mean()
        critical_count = (ved_skus['risk_status'] == 'Critical').sum()
        print(f"   - {ved}: Avg score {avg_score:.1f}, {critical_count} critical")

# 8️⃣ FINAL OUTPUTS FOR TRAINER
print("\n" + "="*70)
print("🎯 TRAINER OUTPUTS")
print("="*70)

print("\n1. 🔹 SCENARIO_SUMMARY.CSV (TOP 10 ROWS):")
print("-" * 70)
print(scenario_summary.head(10).to_string())

print("\n2. 🔹 RISK DISTRIBUTION COUNTS:")
print("-" * 70)
for status, count in risk_distribution.items():
    print(f"   {status}: {count} SKUs")

print("\n3. 🔹 UPDATED GIT TREE:")
print("-" * 70)
import subprocess

# Get git status
status_result = subprocess.run(['git', 'status', '--porcelain'], capture_output=True, text=True)
print("Modified/New files:")
for line in status_result.stdout.strip().split('\n'):
    if line:
        print(f"   {line}")

# Get list of report files
ls_result = subprocess.run(['git', 'ls-files', 'reports/'], capture_output=True, text=True)
report_files = [f for f in ls_result.stdout.strip().split('\n') if f]
print(f"\nTotal report files: {len(report_files)}")

print("\n" + "="*70)
print("✅ DAY 7 SIMULATION & RISK GOVERNANCE ENGINE COMPLETE")
print("="*70)

print("\n📌 KEY GOVERNANCE INSIGHTS:")
print("   • Stock projections simulated for 14-day horizon")
print("   • 4 risk scenarios analyzed: Base, Outbreak, Supply Delay, Worst Case")
print("   • Resilience scores calculated (0-100) with escalation buckets")
print("   • System alerts generated for critical/risky SKUs")
print("   • Ready for operational deployment with risk governance")

🛡️ NHSRC PHC SIMULATION & RISK GOVERNANCE ENGINE
📥 1. Loading Multi-Layer Data Inputs...
   Replenishment data: 12 SKUs
   Time series data: 2,160 records
   Historical range: 2024-01-01 to 2024-06-28

📊 2. Building 14-Day Stock Projection Simulation...
   Running stock projections SKU-by-SKU...
✅ Simulated projections for 12 SKUs

🌍 3. Generating Risk Scenarios...
   Simulating: Base (Normal planning conditions)
   Simulating: Outbreak (Epidemic/seasonal surge (80% demand increase))
   Simulating: Supply_Delay (Transportation disruption)
   Simulating: Worst_Case (Catastrophic risk detection)

🛡️ 4. Calculating Resilience Scores...

🚨 5. Assigning Escalation Buckets...

📤 6. Exporting Simulation Outputs...
   ✅ reports/scenario_summary.csv
   ✅ reports/simulation_dashboard.csv
   ✅ reports/system_alerts.json (8 alerts generated)

📊 7. Risk Distribution Analysis...

   RISK STATUS DISTRIBUTION:
   - Risky: 8 SKUs (66.7%)
   - Watchlist: 4 SKUs (33.3%)

   SCENARIO IMPACT ANALYSIS:
   -